In [ ]:
!npm install -g localtunne

⠙⠹npm error code E404
⠹npm error 404 Not Found - GET https://registry.npmjs.org/localtunne - Not found
⠹npm error 404
⠹npm error 404  'localtunne@*' is not in this registry.
⠹npm error 404
⠹npm error 404 Note that you can also install from a
⠹npm error 404 tarball, folder, http url, or git url.
⠹npm error A complete log of this run can be found in: /root/.npm/_logs/2025-05-09T18_49_25_573Z-debug-0.log
⠹

In [ ]:
!ngrok config add-authtoken 2o8FnuSuioUchcVkQGWsDKHwcRW_28rmkZ86ALGpoVdkgwHx1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install -q streamlit pyngrok lightgbm

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score
import joblib

# Load your dataset (make sure it's uploaded to Colab)
df = pd.read_csv("balanced_features.csv")  # You can upload this using Colab's sidebar
#20k total data (10k-delay/10k-not delay)

# Define features and label
features = ['is_weekend', 'is_peak_hour', 'is_summer_month', 'is_airline_lowcost',
            'departure_month', 'departure_dayofweek', 'taxi_time_ratio']
X = df[features]
y = df['DepDel15']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train LightGBM
lgb_model = LGBMClassifier()
lgb_model.fit(X_train, y_train)

# Evaluate
y_pred = lgb_model.predict(X_test)
y_prob = lgb_model.predict_proba(X_test)[:, 1]

print("🧠 LightGBM Results:")
print(classification_report(y_test, y_pred))
print("✅ ROC AUC:", roc_auc_score(y_test, y_prob))

# Save model
joblib.dump(lgb_model, "lightgbm_model.pkl")
print("✅ Model saved as lightgbm_model.pkl")

[LightGBM] [Info] Number of positive: 7947, number of negative: 8002
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001965 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 267
[LightGBM] [Info] Number of data points in the train set: 15949, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498276 -> initscore=-0.006897
[LightGBM] [Info] Start training from score -0.006897
🧠 LightGBM Results:
              precision    recall  f1-score   support

         0.0       0.65      0.77      0.71      1978
         1.0       0.72      0.60      0.66      2010

    accuracy                           0.68      3988
   macro avg       0.69      0.68      0.68      3988
weighted avg       0.69      0.68      0.68      3988

✅ ROC AUC: 0.735040419741535
✅ Model saved as lightgbm_model.pkl


In [ ]:
streamlit_code = """
import streamlit as st
import pandas as pd
import joblib
from lightgbm import LGBMClassifier

st.set_page_config(page_title="✈️ Flight Delay Predictor")

@st.cache_resource
def load_model():
    return joblib.load("lightgbm_model.pkl")

model = load_model()


st.title("✈️ Flight Delay Predictor")

st.markdown("Fill in the flight details below to predict whether the flight will be delayed.")

st.sidebar.header("Flight Features")

is_weekend = st.sidebar.selectbox("Is Weekend?", [0, 1])
is_peak_hour = st.sidebar.selectbox("Is Peak Hour?", [0, 1])
is_summer_month = st.sidebar.selectbox("Is Summer Month?", [0, 1])
is_airline_lowcost = st.sidebar.selectbox("Is Low-Cost Airline?", [0, 1])
departure_month = st.sidebar.slider("Departure Month", 1, 12, 6)
departure_dayofweek = st.sidebar.slider("Day of Week (0=Mon, 6=Sun)", 0, 6, 0)
taxi_time_ratio = st.sidebar.slider("Taxi Time Ratio", 0.0, 3.0, 1.0)

input_data = pd.DataFrame({
    'is_weekend': [is_weekend],
    'is_peak_hour': [is_peak_hour],
    'is_summer_month': [is_summer_month],
    'is_airline_lowcost': [is_airline_lowcost],
    'departure_month': [departure_month],
    'departure_dayofweek': [departure_dayofweek],
    'taxi_time_ratio': [taxi_time_ratio]
})

if st.button("Predict Delay"):
    prediction = model.predict(input_data)[0]
    prob = model.predict_proba(input_data)[0][1]

    st.subheader("📈 Prediction Result:")
    if prediction == 1:
        st.error(f"The model predicts a **delay** with a probability of {prob:.2f}.")
    else:
        st.success(f"The model predicts **on-time** departure with a probability of {1 - prob:.2f}.")
"""

with open("streamlit.py", "w") as f:
    f.write(streamlit_code)

In [ ]:
import threading, time, os
from pyngrok import ngrok

ngrok.kill()  # stop previous tunnels

def run():
    os.system("streamlit run streamlit.py")

thread = threading.Thread(target=run)
thread.start()
time.sleep(5)

public_url = ngrok.connect(8501)
print(f"🚀 Your Streamlit app is live at: {public_url}")

🚀 Your Streamlit app is live at: NgrokTunnel: "https://55c1-35-199-9-138.ngrok-free.app" -> "http://localhost:8501"
